# 1. Parameters

In [1]:
# Defaults
cases_dir = 'cases/unset'
reference_file = 'references/NC_045512.gbk.gz'
input_files_all = 'input/input-files.tsv'
iterations = 3
mincov = 10
ncores = 32
number_samples = 10
build_tree = False

In [2]:
# Parameters
cases_dir = "cases/case-20000"
iterations = 3
number_samples = 20000
build_tree = False


In [3]:
from pathlib import Path
from shutil import rmtree
from os import makedirs
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)

if cases_dir_path.exists():
    rmtree(cases_dir_path)
    
if not cases_dir_path.exists():
    makedirs(cases_dir_path)

input_files_all = Path(input_files_all)
reference_file = Path(reference_file)

case_name = str(cases_dir_path.name)
reference_name = reference_file.name.split('.')[0]

cases_input = cases_dir_path / 'input-files-case.tsv'
index_path = cases_dir_path / 'index'
benchmark_path = cases_dir_path / 'index-info.tsv'
output_tree = cases_dir_path / 'tree.tre'

# 2. Create subset input

In [4]:
import pandas as pd

all_input_df = pd.read_csv(input_files_all, sep='\t')
all_input_total = len(all_input_df)

subset_input_df = all_input_df.head(number_samples)
subset_input_total = len(subset_input_df)

subset_input_df.to_csv(cases_input, sep='\t', index=False)

print(f'Wrote {subset_input_total}/{all_input_total} samples to {cases_input}')

Wrote 20000/100000 samples to cases/case-20000/input-files-case.tsv


# 2. Index genomes

In [5]:
!gdi --version

gdi, version 0.3.0.dev12


## 2.1. Index reads

In [6]:
results_handler = gdi_benchmark.BenchmarkResultsHandler(name=case_name)
benchmarker = gdi_benchmark.IndexBenchmarker(benchmark_results_handler=results_handler,
                                             index_path=index_path, input_files_file=cases_input,
                                             reference_file=reference_file, mincov=mincov,
                                             build_tree=build_tree,
                                             ncores=ncores)

benchmark_df = benchmarker.benchmark(iterations=iterations)


Iteration 1 of index/analysis of 20000 samples with 32 cores
Removing any extra snakemake directories: ['snakemake-assemblies.1631155580.1447754']


Creating new index: [gdi init cases/case-20000/index]


Creating a new index took 2.82 seconds
Analysis running: [gdi --project-dir cases/case-20000/index --ncores 32 analysis --use-conda --no-load-data --reference-file references/NC_045512.gbk.gz --kmer-size 31 --kmer-size 51 --kmer-size 71 --include-kmer --reads-mincov 10 --input-structured-genomes-file cases/case-20000/input-files-case.tsv]


Analysis took 129.94 minutes
Index running: [gdi --project-dir cases/case-20000/index --ncores 32 load vcf-kmer --reference-file references/NC_045512.gbk.gz /home/CSCScience.ca/apetkau/workspace/genomics-data-index-evaluation/evaluations/sars-cov-2/snakemake-assemblies.1631160136.7038777/gdi-input.fofn]


Indexing took 22.00 minutes


...

Creating a new index took 3.23 seconds
Analysis running: [gdi --project-dir cases/case-20000/index --ncores 32 analysis --use-conda --no-load-data --reference-file references/NC_045512.gbk.gz --kmer-size 31 --kmer-size 51 --kmer-size 71 --include-kmer --reads-mincov 10 --input-structured-genomes-file cases/case-20000/input-files-case.tsv]


Analysis took 129.88 minutes
Index running: [gdi --project-dir cases/case-20000/index --ncores 32 load vcf-kmer --reference-file references/NC_045512.gbk.gz /home/CSCScience.ca/apetkau/workspace/genomics-data-index-evaluation/evaluations/sars-cov-2/snakemake-assemblies.1631178584.174109/gdi-input.fofn]


Indexing took 22.35 minutes


Attempting to set global database_path_translator=<genomics_data_index.storage.model.db.DatabasePathTranslator.DatabasePathTranslator object at 0x7f41a8ac5f10> but it is already set


A reminder to myself to look for a Python solution for directory sizes (instead of running `du`)


In [7]:
benchmark_df

,Name,Iteration,Number samples,Number features (all),Number features (no unknown),Number cores,Reference length,Analysis runtime,Analysis memory (max),Analysis memory (max/process),Analysis disk uage,Index runtime,Index memory (max),Index memory (max/process),Index size,Tree runtime,Tree memory (max),Tree memory (max/process),Total runtime,Max memory
0,case-20000,1,20000,45667,15764,32,29903,7745.0,2.200769e+09,1.908646e+09,7.170032e+09,1319.83,3.614233e+10,2.383262e+09,365797376.0,<NA>,<NA>,<NA>,9064.83,3.614233e+10
0,case-20000,2,20000,45667,15764,32,29903,7863.0,2.199781e+09,1.908212e+09,7.169950e+09,1402.24,3.612909e+10,2.401157e+09,366092288.0,<NA>,<NA>,<NA>,9265.24,3.612909e+10
0,case-20000,3,20000,45667,15764,32,29903,7745.0,2.201899e+09,1.908556e+09,7.169745e+09,1341.05,3.469868e+10,2.359579e+09,366034944.0,<NA>,<NA>,<NA>,9086.05,3.469868e+10


In [8]:
benchmark_df.to_csv(benchmark_path, sep='\t', index=False)

# 3. Export trees

In [9]:
if build_tree:
    !gdi --project-dir {index_path} export tree {reference_name} > {output_tree}
    print(f'Wrote tree to {output_tree}')
else:
    print(f'build_tree={build_tree} so no tree to export')

build_tree=False so no tree to export
